Step 1: Importing libraries


In [231]:
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import LabelEncoder
from decimal import Decimal

Step 2: We will have total 9 files so we will process them sequentially. First lets define column names and data type.

In [232]:
directory = '../datasets/original_dataset'
cleaned_directory = '../datasets/cleaned_data'

column_data_types = {
    'olist_customers_dataset.csv' : {"customer_id": np.str_,
                                     "customer_unique_id": np.str_,
                                     "customer_zip_code_prefix": np.int64,
                                     "customer_city": np.str_,
                                     "customer_state": np.str_},
    'olist_geolocation_dataset.csv' : {"geolocation_zip_code_prefix": np.int64,
                                       "geolocation_lat": Decimal, # for precision
                                       "geolocation_lng": Decimal,
                                       "geolocation_city": np.str_,
                                       "geolocation_state": np.str_},
    'olist_order_items_dataset.csv' : {"order_id": np.str_,
                                       "order_item_id": np.int64,
                                       "product_id": np.str_,
                                       "seller_id": np.str_,
                                       "shipping_limit_date": np.datetime64,
                                       "price": np.float128,
                                       "freight_value": np.float128},
    'olist_order_payments_dataset.csv': {"order_id": np.str_,
                                         "payment_sequential": np.int64,
                                         "payment_type": np.str_,
                                         "payment_installments": np.int64,
                                         "payment_value": np.float128},
    'olist_order_reviews_dataset.csv': {"review_id": np.str_,
                                        "order_id": np.str_,
                                        "review_score": np.float128,
                                        "review_comment_title": np.str_,
                                        "review_comment_message": np.str_,
                                        "review_creation_date": np.datetime64,
                                        "review_answer_timestamp": np.datetime64},
    'olist_orders_dataset.csv' : {"order_id": np.str_,
                                  "customer_id": np.str_,
                                  "order_status": np.str_,
                                  "order_purchase_timestamp": np.datetime64,
                                  "order_approved_at": np.datetime64,
                                  "order_delivered_carrier_date": np.datetime64,
                                  "order_delivered_customer_date": np.datetime64,
                                  "order_estimated_delivery_date": np.datetime64},
    'olist_products_dataset.csv': {"product_id": np.str_,
                                   "product_category_name": np.str_,
                                   "product_name_lenght": np.int64,
                                   "product_description_lenght": np.int64,
                                   "product_photos_qty": np.int64,
                                   "product_weight_g": np.float128,
                                   "product_length_cm": np.float128,
                                   "product_height_cm": np.float128,
                                   "product_width_cm": np.float128},
    'olist_sellers_dataset.csv' : {"seller_id": np.str_,
                                   "seller_zip_code_prefix": np.int64,
                                   "seller_city": np.str_,
                                   "seller_state": np.str_},
    'product_category_name_translation.csv': {'product_category_name': np.str_,
                                              'product_category_name_english': np.str_}
}
primary_key_intro = {
    'olist_customers_dataset.csv' : ["customer_id","customer_unique_id"],
    'olist_geolocation_dataset.csv' : ["geolocation_lat", "geolocation_lng"],
    'olist_order_items_dataset.csv' : ["order_id", "order_item_id"],
    'olist_order_payments_dataset.csv': ["order_id", "payment_sequential"],
    'olist_orders_dataset.csv' : "order_id",
    'olist_products_dataset.csv': "product_id",
    'olist_sellers_dataset.csv' : "seller_id",
    'product_category_name_translation.csv': 'product_category_name'
}

foreign_key_check = {
    'olist_customers_dataset.csv' : [['olist_orders_dataset.csv'], 'customer_id'],
    'olist_geolocation_dataset.csv' : [['olist_customers_dataset.csv', 'olist_sellers_dataset.csv'], 'zip_code_prefix'],
    'olist_order_reviews_dataset.csv': [['olist_orders_dataset.csv'], 'order_id'],
    'olist_products_dataset.csv': [['olist_order_items_dataset.csv'], 'product_id'],
    'olist_sellers_dataset.csv' : [['olist_order_items_dataset.csv'], 'seller_id'],
    'product_category_name_translation.csv': [['olist_products_dataset.csv'],'product_category_name'],
}

pesos_usd = {
    'olist_order_items_dataset.csv': ['price','freight_value'],
    'olist_order_payments_dataset.csv': ['payment_value'],
}

category_labeling = {
    'product_category_name_translation.csv': 'olist_products_dataset.csv'
}

wrong_column_name = {
    'olist_products_dataset.csv': {"product_name_lenght",
                                   "product_description_lenght"},
}

some functions which will process the data.

In [233]:
def pesos_to_usd(pesos):
    return pesos * 0.060

Step 3: Next, we will collect all file paths from the directory and keep it inside a list.

After that, we will process each file at a time, for the assigned data types we will process null values, wrong/duplicate values and transform to input into database.

Finally, we will generate csv file with the cleaned data.

In [234]:
def handle_null_values(df, file):
    
    column_and_types = column_data_types[file]
    
    if column_data_types is not None:
        for col, col_type in column_and_types.items():
            if col_type == np.str_:
                df[col] = df[col].fillna('')
            elif col_type == np.int64 or col_type == np.float128:
                df[col] = df[col].fillna(0)
            elif col_type == np.datetime64:
                df[col] = df[col].fillna(pd.Timestamp('0001-01-01 00:00:00'))
                
    return df

In [235]:
def ensure_foreign_key(df, directory, file):
    
    foreign_key_table = foreign_key_check.get(file)
    
    if foreign_key_table:
        related_files, key_column = foreign_key_table
        
        for related_file in related_files:
            related_file_path = os.path.join(directory, related_file)
            related_df = pd.read_csv(related_file_path)
            
            related_column = next((col for col in related_df.columns if key_column in col), None)
            df_column = next((col for col in df.columns if key_column in col), None)
            
            missing_ids = related_df[~related_df[related_column].isin(df[df_column])][related_column]
            missing_df = pd.DataFrame({related_column: missing_ids}).dropna().drop_duplicates()
           
            df = pd.concat([df, missing_df], ignore_index=True)
    return df

In [236]:
def remove_duplicates(df, file):
    primary_key = primary_key_intro.get(file)
    
    if primary_key:
        if isinstance(primary_key, str):
            primary_key = [primary_key]
        df.drop_duplicates(subset=primary_key, inplace=True)
        
    return df

In [237]:
def convert_currency(df):
    #currency_conversion = pesos_usd.get(file)
    
    if file in pesos_usd:
        for col in pesos_usd[file]:
            df[col] = df[col].apply(pesos_to_usd)
    return df

In [238]:
def encode_categorical(df, file, cleaned_directory):
    category_label = category_labeling.get(file)
    
    if category_label:
        label_encoder = LabelEncoder()
        df = df._append({'product_category_name': 'nocategory','product_category_name_english': 'nocategory'}, ignore_index=True)
        
        df['product_category_id'] = label_encoder.fit_transform(df['product_category_name'])

        product_file = os.path.join(cleaned_directory, category_label)
        product_df = pd.read_csv(product_file)
        
        category_mapping = dict(zip(df['product_category_name'], df['product_category_id']))
        product_df['product_category_id'] = product_df['product_category_name'].fillna('nocategory').map(category_mapping).astype(np.int64)
        product_df.drop(columns=['product_category_name'], inplace=True)
        
        product_df.to_csv(product_file, index=False)
        
    return df

In [239]:
def validating_data(df, file):
    
    #Fix translation Using google API
    
    
    #for ensuring all int, float values > 0
    for col in df.select_dtypes(include=[np.int64, np.float128]).columns:
        if (df[col] < 0).any():
            df[col] = df[col].apply(lambda x: abs(x))
            
    #Filling all empty string with unknown
    for col in df.select_dtypes(include=['object']).columns:
        df[col] = df[col].fillna('Unknown')
        
    #correct wrong column names(manually checked, and changing)
    column_names = wrong_column_name.get(file)
    if column_names:
        for column in column_names:
            correct_column = column.replace('lenght', 'length')
            df.rename(columns={column: correct_column}, inplace=True)
    
    return df

In [240]:
for file in os.listdir(directory):
    if file.endswith('.csv'):
        file_path = os.path.join(directory, file)
        df = pd.read_csv(file_path)
        
         #Foreign key ensuring cross-examination
        df = ensure_foreign_key(df, directory, file)
        
        #Handling null values
        df = handle_null_values(df, file)
        
        #Removing duplicates based on primary key(tokenization)
        df = remove_duplicates(df, file)
        
        #Standardizing & normalizing data (basic currency conversion)
        df = convert_currency(df)
        
        
        #Encoding categorical variables
        df = encode_categorical(df, file, cleaned_directory)
        
        #Missing value fill, validating data, translation, text correction
        df = validating_data(df, file)
            
        df.to_csv(os.path.join(cleaned_directory,file), index=False)

Handling outliers
Handling missing or incorrect or cleanup values more specifically



Handling outliers:

Use descriptive statistics and visualization techniques to identify outliers in your data.
Decide on an appropriate method to handle outliers, such as trimming, winsorizing, or transformation.
Implement the chosen method using NumPy or Pandas.
Standardizing or normalizing data:

Choose a scaling method based on the distribution of your data and the requirements of your analysis.
Use Scikit-learn's preprocessing module to apply min-max scaling (MinMaxScaler) or z-score normalization (StandardScaler) to your numerical features.
Encoding categorical variables:

Convert categorical variables into numerical representations using one-hot encoding or label encoding.
Pandas provides convenient functions like get_dummies() for one-hot encoding and factorize() for label encoding.
Handling missing or incorrect values:

Identify missing or incorrect values in your dataset using descriptive statistics or visualization.
Decide on an appropriate strategy for handling missing values, such as imputation, removal, or correction.
Implement the chosen strategy using Pandas or Scikit-learn's SimpleImputer for imputation or dropna() for removal.